In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq
)
from datasets import Dataset
from tqdm.auto import tqdm 

In [ ]:
# !rm -rf /kaggle/working/*

In [ ]:
class FoodQAModel:
    def __init__(self, model_name='google/flan-t5-small'):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
        
        # Ensure proper padding
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
    def prepare_dataset(self, dataframe, max_length=256):
        # Tokenize the data
        encodings = self.tokenizer(
            dataframe['Question'].tolist(),
            text_target=dataframe['Answer'].tolist(),
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )
        
        # Create Hugging Face Dataset
        dataset = Dataset.from_dict({
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask'],
            'labels': encodings['labels']
        })
        
        return dataset
    
    def train(self, dataset, output_dir='./food_qa_model'):
        # Training configuration
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            overwrite_output_dir=True,
            num_train_epochs=30,
            per_device_train_batch_size=2,
            save_steps=100,
            save_total_limit=2,
            learning_rate=5e-5,
            weight_decay=0.01,
            logging_dir=f'{output_dir}/logs',
            logging_steps=10,
            report_to='tensorboard'
        )
        
        # Data collator
        data_collator = DataCollatorForSeq2Seq(
            self.tokenizer, 
            model=self.model,
            padding=True
        )
        
        # Initialize trainer
        trainer = Seq2SeqTrainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset,
            data_collator=data_collator
        )
        
        # Train the model
        print("Training started...")
        with tqdm(total=training_args.num_train_epochs) as progress_bar:
            for _ in range(training_args.num_train_epochs):
                trainer.train(resume_from_checkpoint=False)
                progress_bar.update(1)
        print("Training completed.")
        
        # Save the model and tokenizer
        trainer.save_model(output_dir)
        self.tokenizer.save_pretrained(output_dir)
        
    def generate_response(self, question, max_length=200):
        # Tokenize the input question
        inputs = self.tokenizer(
            question, 
            return_tensors="pt", 
            truncation=True, 
            max_length=256
        ).to(self.device)
        
        # Generate the response
        outputs = self.model.generate(
            inputs['input_ids'], 
            attention_mask=inputs['attention_mask'],
            max_length=max_length, 
            num_return_sequences=1,
            num_beams=3,
            no_repeat_ngram_size=2,
            top_k=30,
            top_p=0.9,
            temperature=0.7
        )
        
        # Decode and return the response
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response.strip()

In [ ]:
 data = pd.read_csv('/kaggle/input/cleaned-dataset/cleaned_data.csv')

In [ ]:
model_trainer = FoodQAModel()

In [ ]:
train_dataset = model_trainer.prepare_dataset(data)

In [ ]:
model_trainer.train(train_dataset)

In [ ]:
response = model_trainer.generate_response("How to cook Mi quang?")

In [ ]:
print(response)

In [ ]:
from huggingface_hub import login

huggingface_token = "My token" 

login(token=huggingface_token)

In [ ]:
model_trainer.model.save_pretrained('./food-qa-model')

In [ ]:
model_trainer.tokenizer.save_pretrained('./food-qa-model')

In [ ]:
model_trainer.model.push_to_hub("Smrfhdl/DishesQAModel")  
model_trainer.tokenizer.push_to_hub("Smrfhdl/DishesQAModel")